In [1]:
import pandas as pd
import numpy as np

from features import suffix, domains
from fuzzywuzzy import fuzz


c:\Users\LG\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


The idea is to check if link is from a popular domain (or tries to imitate it)
Two features will be added to preprocessing:
- what is the closest levenshtein distance from a popular domain (intution - if it's small but positive, it can be done to imitate the domain)
- does the domain expansion match the closest domain's domain expansion (which might act mean towards diffrent local versions of popular sites)

In [2]:
df = pd.read_csv('OtherData/cloudflare-radar_top-1000-domains_20260119-20260126.csv')
df["suffix"] = suffix(df["domain"])
df["domains"] = domains(df["domain"])

print(df[(df["domains"].apply(len) > 1) | (df["suffix"].apply(len) > 1)][0:20])

                             domain    suffix                          domains
11                         a-mo.net     {net}                          {mo, a}
12                     a-msedge.net     {net}                      {msedge, a}
19                  ad-delivery.net     {net}                   {delivery, ad}
20                     ad-score.com     {com}                      {score, ad}
42                  ads-twitter.com     {com}                   {twitter, ads}
54                      aiv-cdn.net     {net}                       {aiv, cdn}
55                 aiv-delivery.net     {net}                  {aiv, delivery}
68             aliexpress-media.com     {com}              {aliexpress, media}
78              amazon-adsystem.com     {com}               {adsystem, amazon}
79                     amazon.co.jp  {co, jp}                         {amazon}
80                     amazon.co.uk  {co, uk}                         {amazon}
89                amp-endpoint2.com     {com}       

In [3]:
def join_domains(domains_set):
    return "-".join(sorted(list(domains_set)))

df["joined_domains"] = df["domains"].apply(join_domains)
df[(df["domains"].apply(len) > 1) | (df["suffix"].apply(len) > 1)]


,domain,suffix,domains,joined_domains
11,a-mo.net,{net},"{mo, a}",a-mo
12,a-msedge.net,{net},"{msedge, a}",a-msedge
19,ad-delivery.net,{net},"{delivery, ad}",ad-delivery
20,ad-score.com,{com},"{score, ad}",ad-score
42,ads-twitter.com,{com},"{twitter, ads}",ads-twitter
...,...,...,...,...
925,wac-msedge.net,{net},"{msedge, wac}",msedge-wac
968,xnxx-cdn.com,{com},"{xnxx, cdn}",cdn-xnxx
970,xvideos-cdn.com,{com},"{xvideos, cdn}",cdn-xvideos
974,yahoo.co.jp,"{co, jp}",{yahoo},yahoo


In [4]:
df_grouped = df.groupby("joined_domains").agg({
     "suffix": lambda x: set().union(*x),
     "domains": "first",
     "domain": "first",
})

df_grouped

,suffix,domains,domain
joined_domains,,,
,"{ne, jp}",{},ne.jp
1-eu-id5-sync,{com},"{1, eu, id5, sync}",eu-1-id5-sync.com
163,{com},{163},163.com
1drv,{com},{1drv},1drv.com
1rx,{io},{1rx},1rx.io
...,...,...,...
zendesk,{com},{zendesk},zendesk.com
zijieapi,{com},{zijieapi},zijieapi.com
zoho,{com},{zoho},zoho.com


In [9]:
x = fuzz.ratio('google','g00gle')

print(x)

67
